# 6.864 ShaderGenerator Final Project

This notebook contains code that trains the Hearthstone baseline model for the ShaderGenerator final project. We first download our Github repo that contains the datasets we use, install some dependencies, and set up our environment and imports.

In [1]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
rm -rf ShaderGenerator
git clone \
    --depth 1 \
    --filter=blob:none \
    --no-checkout \
    https://github.com/RichardMuri/ShaderGenerator.git
cd ShaderGenerator
git checkout main -- torchASN/ card2code/third_party/hearthstone trained_models/

Cloning into 'ShaderGenerator'...


In [2]:
!pip install torch

In [6]:
import sys
sys.path.append("/content/ShaderGenerator/")
sys.path.append("/content/ShaderGenerator/torchASN/")
import numpy as np
import random
import torch
from torch import cuda
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from dataclasses import dataclass

from common.config import *
from components.dataset import *

from grammar.grammar import Grammar

from grammar.python3.python3_transition_system import Python3TransitionSystem
from models.ASN import ASNParser
from models import nn_utils

from torch import optim
import os
import time
from google.colab import files
import pickle
from google.colab import drive

seed = 0
random.seed(seed)
np.random.seed(seed)

device = None
if cuda.is_available():
  device = 'cuda'
  torch.cuda.manual_seed_all(seed)
else:
  print('WARNING: you are running this project on a cpu!')
  device = 'cpu'
# device = 'cpu'

# Pre-processing Hearthstone Data

In [10]:
# %%bash 
# cd ShaderGenerator/
# export PYTHONPATH="${PYTHONPATH}:torchASN/"
# python torchASN/datasets/hearthstone/make_dataset.py

Traceback (most recent call last):
  File "torchASN/datasets/hearthstone/make_dataset.py", line 109, in <module>
    make_dataset()
  File "torchASN/datasets/hearthstone/make_dataset.py", line 95, in make_dataset
    train_set = load_dataset("train", transition_system)
  File "torchASN/datasets/hearthstone/make_dataset.py", line 68, in load_dataset
    assert transition_system.compare_ast(ast_from_action, tgt_ast)
AssertionError


# Training the Model

We now train the model.

In [12]:
asdl_file = "/content/ShaderGenerator/torchASN/data/hearthstone/python_3_7_12_asdl.txt"
vocab = "/content/ShaderGenerator/torchASN/data/hearthstone/vocab.bin"
train_file = "/content/ShaderGenerator/torchASN/data/hearthstone/train.bin"
dev_file = "/content/ShaderGenerator/torchASN/data/hearthstone/dev.bin"
test_file = "/content/ShaderGenerator/torchASN/data/hearthstone/test.bin"
dropout = 0.3
enc_hid_size = 100
src_emb_size = 100
field_emb_size = 100
max_epoch = 100
clip_grad = 5.0
batch_size = 32
lr = 0.003
model_file = f"model.hearthstone.enc{enc_hid_size}.src{src_emb_size}.field{field_emb_size}.drop{dropout}.max_ep{max_epoch}.batch{batch_size}.lr{lr}.clip_grad{clip_grad}.bin"
save_to = f"/content/ShaderGenerator/torchASN/checkpoints/hearthstone/{model_file}"
log_every = 50
run_val_after = 5
max_decode_step = 70
max_naive_parse_depth = 18
beam_size = 10

@dataclass
class Arguments:
    asdl_file: str
    vocab: str
    train_file: str
    dev_file: str
    test_file: str
    dropout: float
    enc_hid_size: int
    src_emb_size: int
    field_emb_size: int
    max_epoch: int
    clip_grad: float
    batch_size: int
    lr: float
    model_file: str
    save_to: str
    log_every: int
    run_val_after: int
    max_decode_step: int
    max_naive_parse_depth: int
    beam_size: int
    device: str


args = Arguments(asdl_file=asdl_file, vocab=vocab, train_file=train_file, dev_file=dev_file, test_file=test_file, dropout=dropout, enc_hid_size=enc_hid_size, src_emb_size=src_emb_size,
                 field_emb_size=field_emb_size, max_epoch=max_epoch, clip_grad=clip_grad, batch_size=batch_size, lr=lr, model_file=model_file, save_to=save_to, log_every=log_every,
                 run_val_after=run_val_after, max_decode_step=max_decode_step, max_naive_parse_depth=max_naive_parse_depth, beam_size=beam_size, device=device)



# test_set = Dataset.from_bin_file(args.test_file)
# model_path = f"/content/drive/MyDrive/{model_file}"
model_path = "/content/drive/MyDrive/model.hearthstone.bin"


# This will prompt for authorization.
drive.mount('/content/drive')

use_cuda = False
if device == 'cuda':
  use_cuda = True

parser = ASNParser.load(model_path, ex_args=args, cuda=use_cuda)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# parser.eval()
# with torch.no_grad():
#     parse_results = []
#     for ex in tqdm(test_set, desc='Decoding', file=sys.stdout, total=len(test_set)):
#         parse_results.append(parser.parse(ex) )

In [ ]:
with open("parse_results.txt", "wb") as f:
  pickle.dump(parse_results, f)

files.download("parse_results.txt")

In [17]:
path = "content/ShaderGenerator/trained_models/parse_results.txt"
with open(path, "rb") as f:
  parse_results = pickle.load(f)

FileNotFoundError: ignored

In [16]:
# match_results = [ parser.transition_system.compare_ast(r, e.tgt_ast) for r, e in zip(parse_results, test_set)]
# match_acc = sum(match_results) * 1. / len(match_results)
# print("Eval Acc", match_acc)bv
act_tree_to_ast = lambda x: parser.transition_system.build_ast_from_actions(x)
top_asts = [ act_tree_to_ast(x[0].action_tree) if x else None for x in parse_results]
top_codes = [parser.transition_system.ast_to_surface_code(x) for x in top_asts if x is not None]
# match_results = [ parser.transition_system.compare_ast(r, e.tgt_ast) for r, e in zip(top_asts, test_set)]
match_results = [ " ".join(e.tgt_toks) == r for e, r in zip(test_set, top_codes)]
# top_asts = [parser.transition_system]

match_acc = sum(match_results) * 1. / len(match_results)
# [print("%s\n\t==>%s\n\t==>%s" % (" ".join(e.src_toks), " ".join(e.tgt_toks), c)) for e,c in zip(test_set, top_codes)]


with open("output.txt", "w") as f:
    for c in top_codes:
        f.write(c.replace(" ","") + "\n")

files.download("output.txt")

with open("trees.txt", "w") as f:
  for tree in top_asts:
    f.write(top_asts.to_string() + "\n")

files.download("trees.txt")
# oracle_res = []
# i = 0
# acc = 0
# for e, c in zip(test_set, top_codes):
#     gt_code = " ".join(e.tgt_toks)
#     pred_code = c
#     eq_res = check_equiv(pred_code, gt_code)
#     oracle_res.append(eq_res)
#     acc += eq_res
#     i += 1
#     # print(acc, i)
# print("String Acc", match_acc)
# print("DFA Acc", sum(oracle_res) * 1.0/len(oracle_res) )

NameError: ignored

In [14]:
%debug

> /content/ShaderGenerator/torchASN/grammar/python3/py_asdl_helper.py(71)asdl_ast_to_python_ast()
     69 
     70 def asdl_ast_to_python_ast(asdl_ast_node, grammar):
---> 71     py_node_type = getattr(sys.modules['ast'], asdl_ast_node.production.constructor.name)
     72     py_ast_node = py_node_type()
     73 

ipdb> u 12
> <ipython-input-13-5701029343bd>(6)<module>()
      4 act_tree_to_ast = lambda x: parser.transition_system.build_ast_from_actions(x)
      5 top_asts = [ act_tree_to_ast(x[0].action_tree) if x else None for x in parse_results]
----> 6 top_codes = [parser.transition_system.ast_to_surface_code(x) for x in top_asts]
      7 # match_results = [ parser.transition_system.compare_ast(r, e.tgt_ast) for r, e in zip(top_asts, test_set)]
      8 match_results = [ " ".join(e.tgt_toks) == r for e, r in zip(test_set, top_codes)]

ipdb> top_asts.to_string()
*** AttributeError: 'list' object has no attribute 'to_string'
ipdb> top_asts[0].to_string()
*** AttributeError: 'NoneType'


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)

